In [1]:
%pip install rank-bm25 nltk tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install nltk
import nltk
nltk.download('punkt')          # required by word_tokenize/sent_tokenize
try:
    nltk.download('punkt_tab')
except Exception:
    pass


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chrisperez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/chrisperez/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import json
from tqdm import tqdm
import os

In [4]:
chunk_path = "../data/datastore_chunks.jsonl"    
anchors_path = "../data/wiki_qa.jsonl"        
out_path = "../data/retrieved_contexts.jsonl"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

In [5]:
print("Loading chunks...")
chunks = []
with open(chunk_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        chunks.append(obj["chunk"])

Loading chunks...


In [6]:
print("Tokenizing corpus...")
tokenized_corpus = [word_tokenize(c.lower()) for c in tqdm(chunks)]

bm25 = BM25Okapi(tokenized_corpus)
print(f"BM25 index built over {len(chunks):,} chunks.")


Tokenizing corpus...


100%|██████████| 17575/17575 [00:08<00:00, 2069.57it/s]


BM25 index built over 17,575 chunks.


In [7]:
anchors = []
with open(anchors_path, "r", encoding="utf-8") as f:
    for line in f:
        anchors.append(json.loads(line)["question"])

In [8]:
k = 1
print(f"Retrieving top-{k} contexts per question...")
with open(out_path, "w", encoding="utf-8") as fout:
    for q in tqdm(anchors):
        tokenized_query = word_tokenize(q.lower())
        scores = bm25.get_scores(tokenized_query)
        topk_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
        context = "\n".join(chunks[i] for i in topk_idx)
        fout.write(json.dumps({"anchor": q, "indices": topk_idx, "context": context}, ensure_ascii=False) + "\n")

print(f"Saved retrieved contexts to {out_path}")

Retrieving top-1 contexts per question...


100%|██████████| 230/230 [00:12<00:00, 17.78it/s]

Saved retrieved contexts to ../data/retrieved_contexts.jsonl
